# Geometry Preprocessing with GMSHFlow

This notebook focuses on GMSHFlow's geometry preprocessing capabilities, which are useful even without GMSH installation. These tools help prepare and clean geospatial data before mesh generation.

Topics covered:
1. Loading and cleaning geospatial data
2. Geometry simplification while preserving topology
3. Processing MultiLineStrings and complex geometries
4. Data validation and quality checks
5. Preparing data for mesh generation
6. Integration with common GIS workflows

**Note**: This example focuses on preprocessing utilities and doesn't require GMSH installation.

In [ ]:
# Import required libraries
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, LineString, MultiLineString, Point, MultiPolygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import gmshflow
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print(f"GMSHFlow version: {gmshflow.__version__}")
print("Available preprocessing functions:")
preprocessing_funcs = [name for name in dir(gmshflow) if 'simplify' in name.lower() or 'merge' in name.lower()]
for func in preprocessing_funcs:
    print(f"  - gmshflow.{func}")

## Step 1: Create Complex Test Geometries

Let's create some complex geometries that represent common challenges in geospatial data preprocessing.

In [ ]:
# Create a complex, high-resolution polygon (simulating detailed survey data)
np.random.seed(42)

# Create a detailed boundary with many vertices
n_points = 200
theta = np.linspace(0, 2*np.pi, n_points)
base_radius = 500

# Add multiple frequency components to create a complex shape
noise = (50 * np.sin(5*theta) + 
         30 * np.sin(12*theta) + 
         20 * np.sin(25*theta) +
         10 * np.random.randn(n_points))

radius = base_radius + noise
x_coords = 1000 + radius * np.cos(theta)
y_coords = 500 + radius * np.sin(theta)

# Create the detailed polygon
detailed_coords = list(zip(x_coords, y_coords))
detailed_polygon = Polygon(detailed_coords)

# Create GeoDataFrame
detailed_gdf = gpd.GeoDataFrame(
    {'name': ['detailed_boundary'], 'type': ['survey_data']}, 
    geometry=[detailed_polygon]
)

print(f"Original detailed polygon: {len(detailed_polygon.exterior.coords)} vertices")
print(f"Area: {detailed_polygon.area:.0f} m²")
print(f"Perimeter: {detailed_polygon.length:.0f} m")

## Step 2: Create Complex LineString Data

Create complex line data that might represent rivers, roads, or fault lines.

In [ ]:
# Create multiple detailed LineStrings
def create_detailed_line(start_point, end_point, n_points=50, noise_amplitude=20):
    """Create a detailed line with noise between two points."""
    t = np.linspace(0, 1, n_points)
    
    # Linear interpolation between start and end
    x_base = start_point[0] + t * (end_point[0] - start_point[0])
    y_base = start_point[1] + t * (end_point[1] - start_point[1])
    
    # Add noise perpendicular to the line direction
    direction = np.array(end_point) - np.array(start_point)
    perpendicular = np.array([-direction[1], direction[0]])
    perpendicular = perpendicular / np.linalg.norm(perpendicular)
    
    noise = noise_amplitude * np.sin(10 * np.pi * t) * (1 + 0.5 * np.random.randn(n_points))
    
    x_coords = x_base + noise * perpendicular[0]
    y_coords = y_base + noise * perpendicular[1]
    
    return LineString(zip(x_coords, y_coords))

# Create several detailed LineStrings
lines = [
    create_detailed_line((400, 300), (1600, 700), n_points=80),  # Main river
    create_detailed_line((600, 100), (1200, 900), n_points=60),  # Tributary 1
    create_detailed_line((800, 200), (1400, 500), n_points=40),  # Tributary 2
    create_detailed_line((500, 600), (1500, 400), n_points=70),  # Road
]

# Create a MultiLineString (common in GIS data)
multi_linestring = MultiLineString(lines)

# Create GeoDataFrame with mixed geometry types
line_data = {
    'name': ['main_river', 'tributary_1', 'tributary_2', 'road', 'multi_feature'],
    'type': ['river', 'river', 'river', 'road', 'mixed'],
    'priority': ['high', 'medium', 'medium', 'low', 'high']
}

geometries = lines + [multi_linestring]
lines_gdf = gpd.GeoDataFrame(line_data, geometry=geometries)

print("Created complex line geometries:")
for idx, row in lines_gdf.iterrows():
    geom = row.geometry
    if isinstance(geom, LineString):
        vertex_count = len(geom.coords)
    elif isinstance(geom, MultiLineString):
        vertex_count = sum(len(line.coords) for line in geom.geoms)
    else:
        vertex_count = "unknown"
    
    print(f"  {row['name']}: {type(geom).__name__}, {vertex_count} vertices")

## Step 3: Visualize Original Complex Geometries

Let's see what our complex, high-resolution data looks like.

In [ ]:
# Create visualization of original complex data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Detailed polygon
detailed_gdf.plot(ax=ax1, alpha=0.3, color='lightblue', edgecolor='blue', linewidth=1)
ax1.set_title(f'Original Detailed Polygon\n({len(detailed_polygon.exterior.coords)} vertices)')
ax1.set_xlabel('X coordinate (m)')
ax1.set_ylabel('Y coordinate (m)')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: Complex line data
color_map = {'river': 'blue', 'road': 'red', 'mixed': 'purple'}
for geom_type in ['river', 'road', 'mixed']:
    subset = lines_gdf[lines_gdf['type'] == geom_type]
    if len(subset) > 0:
        subset.plot(ax=ax2, color=color_map[geom_type], linewidth=2, 
                   alpha=0.7, label=geom_type.title())

ax2.set_title('Original Complex Line Data')
ax2.set_xlabel('X coordinate (m)')
ax2.set_ylabel('Y coordinate (m)')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

print("Original data complexity:")
total_polygon_vertices = len(detailed_polygon.exterior.coords)
total_line_vertices = sum(
    len(geom.coords) if isinstance(geom, LineString) 
    else sum(len(line.coords) for line in geom.geoms)
    for geom in lines_gdf.geometry
)

print(f"  - Polygon vertices: {total_polygon_vertices}")
print(f"  - Line vertices: {total_line_vertices}")
print(f"  - Total vertices: {total_polygon_vertices + total_line_vertices}")

## Step 4: Apply Geometry Simplification

Now let's use GMSHFlow's preprocessing functions to simplify the geometries while preserving their essential characteristics.

In [ ]:
# Test different simplification tolerances
tolerances = [5, 10, 25, 50]
simplified_polygons = {}
simplified_stats = {}

print("Testing polygon simplification with different tolerances:")
print(f"Original: {len(detailed_polygon.exterior.coords)} vertices")

for tolerance in tolerances:
    # Use GMSHFlow's topology-preserving simplification
    # Create a temporary GeoDataFrame with the tolerance as 'cs' column
    temp_gdf = gpd.GeoDataFrame(
        {'cs': [tolerance]}, 
        geometry=[detailed_polygon]
    )
    simplified_gdf = gmshflow.simplify_keeping_topology(temp_gdf,cs=tolerance)
    simplified_geom = simplified_gdf.geometry.iloc[0]
    
    simplified_polygons[tolerance] = simplified_geom
    
    # Calculate statistics
    original_vertices = len(detailed_polygon.exterior.coords)
    simplified_vertices = len(simplified_geom.exterior.coords)
    reduction = (1 - simplified_vertices / original_vertices) * 100
    area_change = abs(simplified_geom.area - detailed_polygon.area) / detailed_polygon.area * 100
    
    simplified_stats[tolerance] = {
        'vertices': simplified_vertices,
        'reduction': reduction,
        'area_change': area_change
    }
    
    print(f"  Tolerance {tolerance}m: {simplified_vertices} vertices "
          f"({reduction:.1f}% reduction, {area_change:.2f}% area change)")

# Choose optimal tolerance (balance between simplification and quality)
optimal_tolerance = 25  # Good balance for this example
simplified_polygon = simplified_polygons[optimal_tolerance]
simplified_polygon_gdf = gpd.GeoDataFrame(
    {'name': ['simplified_boundary'], 'type': ['processed']}, 
    geometry=[simplified_polygon]
)

## Step 5: Process Complex LineString Data

Now let's process the line data, including merging MultiLineStrings.

In [ ]:
# Process MultiLineString data
print("Processing MultiLineString data:")

# Find MultiLineString geometries
multi_line_mask = lines_gdf.geometry.apply(lambda x: isinstance(x, MultiLineString))
multi_lines = lines_gdf[multi_line_mask]

print(f"Found {len(multi_lines)} MultiLineString geometries")

# Process each MultiLineString
processed_lines = []

for idx, row in lines_gdf.iterrows():
    geom = row.geometry
    
    if isinstance(geom, MultiLineString):
        print(f"\nProcessing {row['name']} (MultiLineString with {len(geom.geoms)} parts):")
        
        # Use GMSHFlow's function to merge MultiLineString into single LineString
        try:
            merged_line = gmshflow.merge_many_multilinestring_into_one_linestring([geom])
            if merged_line and len(merged_line) > 0:
                processed_geom = merged_line[0]  # Take the first (and should be only) result
                print(f"  Successfully merged into single LineString")
                print(f"  Original parts: {len(geom.geoms)}")
                print(f"  Merged vertices: {len(processed_geom.coords)}")
            else:
                processed_geom = geom  # Keep original if merging fails
                print(f"  Merging failed, keeping original")
        except Exception as e:
            print(f"  Error during merging: {e}")
            processed_geom = geom  # Keep original on error
    
    elif isinstance(geom, LineString):
        # Simplify regular LineStrings
        original_vertices = len(geom.coords)
        
        # Create a temporary GeoDataFrame with the geometry and cs column
        temp_gdf = gpd.GeoDataFrame(
            {'cs': [10]}, 
            geometry=[geom]
        )
        simplified_gdf = gmshflow.simplify_keeping_topology(temp_gdf,cs=10)
        processed_geom = simplified_gdf.geometry.iloc[0]
        simplified_vertices = len(processed_geom.coords)
        
        print(f"Simplified {row['name']}: {original_vertices} -> {simplified_vertices} vertices")
    
    else:
        processed_geom = geom  # Keep other geometry types as-is
    
    # Create processed row
    processed_row = row.copy()
    processed_row.geometry = processed_geom
    processed_lines.append(processed_row)

# Create processed lines GeoDataFrame
processed_lines_gdf = gpd.GeoDataFrame(processed_lines)

print(f"\nProcessing complete: {len(processed_lines_gdf)} geometries processed")

## Step 6: Visualize Before and After Comparison

Let's create detailed comparisons showing the effect of our preprocessing.

In [ ]:
# Create comprehensive before/after comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original polygon with vertex density visualization
detailed_gdf.plot(ax=ax1, alpha=0.3, color='lightblue', edgecolor='blue', linewidth=2)
# Show vertices as points for the boundary
boundary_coords = np.array(detailed_polygon.exterior.coords)
ax1.scatter(boundary_coords[:, 0], boundary_coords[:, 1], 
           c='red', s=1, alpha=0.5, label='Vertices')
ax1.set_title(f'Original Polygon\n{len(boundary_coords)} vertices')
ax1.set_xlabel('X coordinate (m)')
ax1.set_ylabel('Y coordinate (m)')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_aspect('equal')

# Plot 2: Simplified polygon
simplified_polygon_gdf.plot(ax=ax2, alpha=0.3, color='lightgreen', edgecolor='green', linewidth=2)
simplified_coords = np.array(simplified_polygon.exterior.coords)
ax2.scatter(simplified_coords[:, 0], simplified_coords[:, 1], 
           c='red', s=3, alpha=0.8, label='Vertices')
ax2.set_title(f'Simplified Polygon (tolerance={optimal_tolerance}m)\n{len(simplified_coords)} vertices')
ax2.set_xlabel('X coordinate (m)')
ax2.set_ylabel('Y coordinate (m)')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_aspect('equal')

# Plot 3: Original lines with vertex density
for geom_type in ['river', 'road', 'mixed']:
    subset = lines_gdf[lines_gdf['type'] == geom_type]
    if len(subset) > 0:
        subset.plot(ax=ax3, color=color_map[geom_type], linewidth=3, 
                   alpha=0.7, label=f'Original {geom_type}')

ax3.set_title('Original Line Data\n(High vertex density)')
ax3.set_xlabel('X coordinate (m)')
ax3.set_ylabel('Y coordinate (m)')
ax3.grid(True, alpha=0.3)
ax3.legend()
ax3.set_aspect('equal')

# Plot 4: Processed lines
for geom_type in ['river', 'road', 'mixed']:
    subset = processed_lines_gdf[processed_lines_gdf['type'] == geom_type]
    if len(subset) > 0:
        subset.plot(ax=ax4, color=color_map[geom_type], linewidth=3, 
                   alpha=0.7, label=f'Processed {geom_type}')

ax4.set_title('Processed Line Data\n(Simplified and merged)')
ax4.set_xlabel('X coordinate (m)')
ax4.set_ylabel('Y coordinate (m)')
ax4.grid(True, alpha=0.3)
ax4.legend()
ax4.set_aspect('equal')

plt.tight_layout()
plt.show()

# Create output directory and save
output_dir = Path("./output")
output_dir.mkdir(exist_ok=True)
fig.savefig(output_dir / 'preprocessing_comparison.png', dpi=150, bbox_inches='tight')
print(f"Comparison plot saved to: {output_dir / 'preprocessing_comparison.png'}")

## Step 7: Quantitative Analysis and Quality Metrics

Let's analyze the quantitative impact of our preprocessing.

In [ ]:
# Comprehensive preprocessing analysis
print("=== PREPROCESSING ANALYSIS ===")

# Polygon analysis
print("\n--- POLYGON SIMPLIFICATION ---")
original_poly_vertices = len(detailed_polygon.exterior.coords)
simplified_poly_vertices = len(simplified_polygon.exterior.coords)
poly_reduction = (1 - simplified_poly_vertices / original_poly_vertices) * 100
area_preservation = (1 - abs(simplified_polygon.area - detailed_polygon.area) / detailed_polygon.area) * 100

print(f"Original vertices: {original_poly_vertices}")
print(f"Simplified vertices: {simplified_poly_vertices}")
print(f"Vertex reduction: {poly_reduction:.1f}%")
print(f"Area preservation: {area_preservation:.2f}%")
print(f"Simplification tolerance: {optimal_tolerance}m")

# Line analysis
print("\n--- LINE PROCESSING ---")
original_line_vertices = 0
processed_line_vertices = 0
multilinestring_count = 0
merged_count = 0

for orig_geom, proc_geom in zip(lines_gdf.geometry, processed_lines_gdf.geometry):
    # Count original vertices
    if isinstance(orig_geom, LineString):
        original_line_vertices += len(orig_geom.coords)
    elif isinstance(orig_geom, MultiLineString):
        multilinestring_count += 1
        original_line_vertices += sum(len(line.coords) for line in orig_geom.geoms)
        if isinstance(proc_geom, LineString):  # Successfully merged
            merged_count += 1
    
    # Count processed vertices
    if isinstance(proc_geom, LineString):
        processed_line_vertices += len(proc_geom.coords)
    elif isinstance(proc_geom, MultiLineString):
        processed_line_vertices += sum(len(line.coords) for line in proc_geom.geoms)

line_reduction = (1 - processed_line_vertices / original_line_vertices) * 100

print(f"Original line vertices: {original_line_vertices}")
print(f"Processed line vertices: {processed_line_vertices}")
print(f"Line vertex reduction: {line_reduction:.1f}%")
print(f"MultiLineStrings processed: {multilinestring_count}")
print(f"Successfully merged: {merged_count}")

# Overall analysis
print("\n--- OVERALL IMPACT ---")
total_original = original_poly_vertices + original_line_vertices
total_processed = simplified_poly_vertices + processed_line_vertices
total_reduction = (1 - total_processed / total_original) * 100

print(f"Total original vertices: {total_original}")
print(f"Total processed vertices: {total_processed}")
print(f"Overall vertex reduction: {total_reduction:.1f}%")

# Create summary DataFrame
summary_data = {
    'Metric': [
        'Polygon vertices (original)',
        'Polygon vertices (simplified)',
        'Line vertices (original)',
        'Line vertices (processed)',
        'Total vertices (original)',
        'Total vertices (processed)',
        'Polygon reduction (%)',
        'Line reduction (%)',
        'Overall reduction (%)',
        'Area preservation (%)'
    ],
    'Value': [
        original_poly_vertices,
        simplified_poly_vertices,
        original_line_vertices,
        processed_line_vertices,
        total_original,
        total_processed,
        f"{poly_reduction:.1f}",
        f"{line_reduction:.1f}",
        f"{total_reduction:.1f}",
        f"{area_preservation:.2f}"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n--- SUMMARY TABLE ---")
print(summary_df.to_string(index=False))

## Step 8: Export Processed Data

Save the processed geometries for use in mesh generation or other applications.

In [ ]:
# Export processed data
print("Exporting processed geometries...")

# Save simplified polygon
simplified_polygon_gdf.to_file(output_dir / 'simplified_polygon.shp')
print(f"Simplified polygon saved to: {output_dir / 'simplified_polygon.shp'}")

# Save processed lines
processed_lines_gdf.to_file(output_dir / 'processed_lines.shp')
print(f"Processed lines saved to: {output_dir / 'processed_lines.shp'}")

# Save summary statistics
summary_df.to_csv(output_dir / 'preprocessing_summary.csv', index=False)
print(f"Summary statistics saved to: {output_dir / 'preprocessing_summary.csv'}")

# Create a combined dataset ready for mesh generation
print("\nCreating mesh-ready dataset...")

# Combine all processed geometries
mesh_ready_data = {
    'geometry_type': ['domain'] + ['line'] * len(processed_lines_gdf),
    'name': ['main_domain'] + processed_lines_gdf['name'].tolist(),
    'mesh_priority': ['medium'] + processed_lines_gdf['priority'].tolist()
}

all_geometries = [simplified_polygon] + processed_lines_gdf.geometry.tolist()
mesh_ready_gdf = gpd.GeoDataFrame(mesh_ready_data, geometry=all_geometries)

# Add suggested mesh sizes based on geometry type and priority
mesh_size_map = {
    ('domain', 'medium'): 50,
    ('line', 'high'): 20,
    ('line', 'medium'): 30,
    ('line', 'low'): 40
}

mesh_ready_gdf['suggested_mesh_size'] = [
    mesh_size_map.get((row['geometry_type'], row['mesh_priority']), 50)
    for _, row in mesh_ready_gdf.iterrows()
]

mesh_ready_gdf.to_file(output_dir / 'mesh_ready_geometries.gpkg', driver="GPKG")
print(f"Mesh-ready dataset saved to: {output_dir / 'mesh_ready_geometries.gpkg'}")

print(f"\nMesh-ready dataset contains {len(mesh_ready_gdf)} geometries:")
for geom_type in mesh_ready_gdf['geometry_type'].unique():
    count = len(mesh_ready_gdf[mesh_ready_gdf['geometry_type'] == geom_type])
    print(f"  - {geom_type}: {count} geometries")

## Step 9: Validation and Quality Checks

Perform validation checks on the processed geometries to ensure they're suitable for mesh generation.

In [ ]:
# Comprehensive validation of processed geometries
print("=== GEOMETRY VALIDATION ===")

def validate_geometry(gdf, name):
    """Perform comprehensive validation on a GeoDataFrame."""
    print(f"\n--- {name.upper()} VALIDATION ---")
    
    # Basic checks
    print(f"Number of geometries: {len(gdf)}")
    print(f"CRS defined: {gdf.crs is not None}")
    
    # Geometry validity
    valid_geoms = gdf.geometry.is_valid.sum()
    invalid_geoms = len(gdf) - valid_geoms
    print(f"Valid geometries: {valid_geoms}")
    print(f"Invalid geometries: {invalid_geoms}")
    
    if invalid_geoms > 0:
        print("Warning: Invalid geometries detected!")
        invalid_idx = gdf[~gdf.geometry.is_valid].index
        for idx in invalid_idx:
            print(f"  Invalid geometry at index {idx}: {gdf.loc[idx, 'name'] if 'name' in gdf.columns else 'unnamed'}")
    
    # Geometry types
    geom_types = gdf.geometry.geom_type.value_counts()
    print("Geometry types:")
    for geom_type, count in geom_types.items():
        print(f"  - {geom_type}: {count}")
    
    # Area/length statistics (where applicable)
    if any(gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])):
        polygon_mask = gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])
        areas = gdf[polygon_mask].geometry.area
        if len(areas) > 0:
            print(f"Polygon areas - Min: {areas.min():.0f}, Max: {areas.max():.0f}, Mean: {areas.mean():.0f}")
    
    if any(gdf.geometry.geom_type.isin(['LineString', 'MultiLineString'])):
        line_mask = gdf.geometry.geom_type.isin(['LineString', 'MultiLineString'])
        lengths = gdf[line_mask].geometry.length
        if len(lengths) > 0:
            print(f"Line lengths - Min: {lengths.min():.0f}, Max: {lengths.max():.0f}, Mean: {lengths.mean():.0f}")
    
    # Check for empty geometries
    empty_geoms = gdf.geometry.is_empty.sum()
    if empty_geoms > 0:
        print(f"Warning: {empty_geoms} empty geometries detected!")
    
    # Bounding box
    bounds = gdf.total_bounds
    print(f"Bounding box: ({bounds[0]:.0f}, {bounds[1]:.0f}) to ({bounds[2]:.0f}, {bounds[3]:.0f})")
    
    return valid_geoms == len(gdf) and empty_geoms == 0

# Validate all processed datasets
poly_valid = validate_geometry(simplified_polygon_gdf, "Simplified Polygon")
lines_valid = validate_geometry(processed_lines_gdf, "Processed Lines")
combined_valid = validate_geometry(mesh_ready_gdf, "Mesh-Ready Dataset")

# Overall validation result
print("\n=== OVERALL VALIDATION RESULT ===")
all_valid = poly_valid and lines_valid and combined_valid

if all_valid:
    print("✅ ALL VALIDATIONS PASSED")
    print("Geometries are ready for mesh generation!")
else:
    print("❌ VALIDATION ISSUES DETECTED")
    print("Please review and fix invalid geometries before mesh generation.")

# Mesh generation readiness checklist
print("\n=== MESH GENERATION READINESS ===")
checklist = {
    "All geometries are valid": all_valid,
    "No empty geometries": True,  # Checked in validation
    "Reasonable vertex count": total_processed < 1000,  # Heuristic threshold
    "Geometries within reasonable bounds": True,  # Could add specific checks
    "Domain polygon available": len(simplified_polygon_gdf) > 0,
    "Suggested mesh sizes assigned": 'suggested_mesh_size' in mesh_ready_gdf.columns
}

for check, passed in checklist.items():
    status = "✅" if passed else "❌"
    print(f"{status} {check}")

mesh_ready = all(checklist.values())
print(f"\nMesh generation readiness: {'✅ READY' if mesh_ready else '❌ NOT READY'}")

## Summary

This notebook demonstrated GMSHFlow's powerful geometry preprocessing capabilities:

### Key Preprocessing Functions Used:
1. **`gmshflow.simplify_keeping_topology()`** - Reduces vertex count while preserving geometry shape and area
2. **`gmshflow.merge_many_multilinestring_into_one_linestring()`** - Converts complex MultiLineString to simpler LineString

### Benefits Achieved:
- **Significant vertex reduction** while maintaining geometric accuracy
- **Simplified data structures** easier to work with in mesh generation
- **Quality validation** ensures data integrity before mesh generation
- **Standardized formats** ready for GMSH processing

### Real-World Applications:
- **Survey Data Cleaning**: Simplify high-resolution GPS survey data
- **GIS Data Preparation**: Process complex shapefiles for modeling
- **CAD Data Integration**: Convert detailed CAD geometries for simulation
- **Multi-Source Data**: Combine and clean data from different sources

### Key Takeaways:
1. **Topology preservation** is crucial - simple buffering/simplification can break geometries
2. **Balance is important** - too much simplification loses important features
3. **Validation is essential** - always check geometry validity after processing
4. **Mesh sizing** should be planned during preprocessing, not after

### Next Steps:
- Use the processed geometries in mesh generation workflows
- Experiment with different simplification tolerances for your data
- Integrate preprocessing into automated data pipelines
- Combine with other GIS tools for comprehensive workflows

**Note**: These preprocessing tools work independently of GMSH, making them valuable for any geospatial data workflow, not just mesh generation!